In [46]:
import pandas as pd
from underthesea import word_tokenize
import re

### Hàm xử lý văn bản bằng cách chia câu thành token

In [47]:
def preprocess_text(text):
    return word_tokenize(str(text), format='text').strip().split()

### Hàm xử lý từ khóa bằng cách loại bỏ khoảng trắng và chuyển hết về chữ thường 

In [48]:
def preprocess_keywords(keywords):
    keyword_tokens = word_tokenize(str(keywords), format='text').strip().split()
    return set([token.lower() for token in keyword_tokens if token.strip()])

### Hàm loại bỏ các dấu câu để giảm nhiễu

In [49]:
def is_valid_token(token):
    return re.match(r'^[\wÀ-ỹ0-9_]+$', token) is not None

### Hàm tạo đặc trưng từ token

In [50]:
def token_features(token):
    return {
        'token': token,
        'length': len(token),
        'is_capitalized': int(token[0].isupper()),
        'is_all_caps': int(token.isupper()),
        'has_digit': int(any(c.isdigit() for c in token))
    }


### Hàm tổng hợp tất cả: từ preprocessing đến vector hóa

In [51]:
def preprocess_and_vectorize(df):
    rows = []

    for idx, row in df.iterrows():
        text = row['content']
        keywords = row['keywords']

        tokens = preprocess_text(text)
        keyword_set = preprocess_keywords(keywords)

        for token in tokens:
            if not is_valid_token(token):
                continue
            label = 1 if token.lower() in keyword_set else 0
            feats = token_features(token)
            feats['label'] = label
            rows.append(feats)

    return pd.DataFrame(rows)

In [52]:
df = pd.read_csv('../data/dataset_ml.csv')

In [53]:
df.head()

,keywords,content
0,"mẫu, xét nghiệm, sân bay, Nội Bài, ngừng lấy m...",Thứ trưởng Bộ Y tế Nguyễn Thanh Long tối 18/3 ...
1,"cách ly y tế, giám sát, nghi nhiễm, xét nghiệm",Con số nghi nhiễm giảm 4 trường hợp so với hôm...
2,"thuốc điều trị nCoV, thuốc cúm Avigan, thuốc t...","Ông Zhang Xinmin, Giám đốc Trung tâm phát triể..."
3,"viêm phổi Trung Quốc, Covid-19, nCoV, virus co...","Nếu nghiên cứu thành công, đây sẽ là một kỳ tí..."
4,"viêm phổi Trung Quốc, viêm phổi Vũ Hán, Covid-...","Hồi tháng giêng khi dịch mới bùng phát, hàng n..."


In [54]:
df['keywords'][0]

'mẫu, xét nghiệm, sân bay, Nội Bài, ngừng lấy mẫu, cách ly, Covid-19'

In [55]:
features_df = preprocess_and_vectorize(df)

In [56]:
features_df.head(50)

,token,length,is_capitalized,is_all_caps,has_digit,label
0,Thứ_trưởng,10,1,0,0,0
1,Bộ,2,1,0,0,0
2,Y_tế,4,1,0,0,0
3,Nguyễn_Thanh_Long,17,1,0,0,0
4,tối,3,0,0,0,0
5,cho,3,0,0,0,0
6,biết,4,0,0,0,0
7,nhằm,4,0,0,0,0
8,tháo_gỡ,7,0,0,0,0
9,ách_tắc,7,0,0,0,0


In [57]:
len(features_df)

1898301

In [58]:
sum(features_df['label'] == 1)

133845